
# K-IFRS JSON 전처리 파이프라인 (End-to-End)

이 노트북은 **처음부터 끝까지** 아래 규칙을 적용해 하나의 정리된 JSON을 만듭니다.

## 전처리 규칙 (최신 합본)
1. `text` 내부의 `[...]` 중 **`삭제됨`/`삭제함`**이 들어간 구절은 **그 부분만 삭제** (나머지 문장은 유지).  
2. `text`에 **句點 `.`이 하나도 없으면** 그 문단(para_id, page, text)을 **통째 삭제**.  
3. `text`의 **마지막 `.` 이후 꼬리**가 있으면 검사:  
   - 아래 **허용 표식**이 **하나라도 포함**되면 꼬리 **유지**  
     - 콜론 **`:` 또는 `：`**  
     - 특수 번호 기호 **`⑴–⑽`, `①–⑳`, `㉠–㉿`**  
     - 괄호 번호/문자 시작 **`(1)`, `(A)`, `(가)`** (영문/한글/CJK 모두)  
   - 위 표식이 **전혀 없으면** → **꼬리만 삭제** (`.`까지만 남김)  
4. 전처리 후 `text` **길이 ≤ 10자**면 해당 문단을 **삭제**.  

> 이 규칙은 “정의/목록” 같은 한국어 회계기준서 스타일을 보존하면서, 불필요한 조각 꼬리만 잘라내도록 설계되었습니다.


In [ ]:

from pathlib import Path
import json, re

# ===== 경로 설정 =====
# 아래 경로를 원본/출력 파일로 바꿔서 사용하세요.
INPUT_PATH = Path('.../raws/raws_K-ifrs/kifrs_combined_2.json')   # 원본 결합 JSON 경로 (예시값)
OUTPUT_PATH = Path('.../raws/raws_K-ifrs/kifrs_cleaned_final.json') # 최종 저장 경로

print('INPUT_PATH :', INPUT_PATH)
print('OUTPUT_PATH:', OUTPUT_PATH)


In [ ]:

def load_json(path: Path):
    with path.open(encoding='utf-8') as f:
        return json.load(f)

data = load_json(INPUT_PATH)
assert 'documents' in data, "루트에 'documents' 키가 있어야 합니다."
print('Loaded documents:', len(data['documents']))


In [ ]:

# ===== 패턴 정의 =====
# 1) 부분 삭제: [ ... 삭제됨 ] / [ ... 삭제함 ]
DEL_BRACKET_PATTERN = re.compile(r"\[[^\]]*삭제(?:됨|함)\]", re.UNICODE)

# 2) 문장 존재 여부: 간단히 '.' 포함 여부로 판단
def has_sentence(text: str) -> bool:
    return '.' in text

# 3) 꼬리 허용 표식
COLON_ANY = re.compile(r":|：")  # 콜론 유니코드 포함
CIRCLED_ANY = re.compile(r"[⑴-⑽①-⑳㉠-㉿]")  # 특수 번호 기호
PAREN_NUMBER_START = re.compile(r"^\s*[\(（]\s*\d+\s*[\)）]")  # (1), （1） 등
PAREN_LETTER_START = re.compile(r"^\s*[\(（]\s*[A-Za-z가-힣一-龥]\s*[\)）]")  # (A)/(가)/(一) 등

def allow_tail(tail: str) -> bool:
    # 꼬리에 콜론/특수기호가 '어디든' 포함되면 허용
    if COLON_ANY.search(tail):
        return True
    if CIRCLED_ANY.search(tail):
        return True
    # 괄호 번호/문자는 '시작'에 있으면 허용
    if PAREN_NUMBER_START.search(tail):
        return True
    if PAREN_LETTER_START.search(tail):
        return True
    return False


In [ ]:

from copy import deepcopy

orig = deepcopy(data)
removed_no_sentence = 0
removed_short = 0
removed_empty_after_cut = 0
trimmed_tail = 0
modified_del_marks = 0

for doc in data.get('documents', []):
    new_paras = []
    for p in doc.get('paragraphs', []):
        text = (p.get('text') or '')
        # (1) 부분 삭제: [ ... 삭제됨/삭제함 ] -> 부분 문자열 제거
        new_text = DEL_BRACKET_PATTERN.sub("", text)
        if new_text != text:
            modified_del_marks += 1
        # 공백 정리
        new_text = re.sub(r"\s{2,}", " ", new_text).strip()

        # (2) 부분 삭제 후 완전 비면 drop
        if not new_text:
            removed_empty_after_cut += 1
            continue

        # (3) 문장('.') 없는 문단 drop
        if not has_sentence(new_text):
            removed_no_sentence += 1
            continue

        # (4) 마지막 '.' 뒤 꼬리 처리
        last_dot = new_text.rfind('.')
        if last_dot != -1 and last_dot < len(new_text) - 1:
            tail = new_text[last_dot+1:]
            if tail and not allow_tail(tail):
                new_text2 = new_text[:last_dot+1].rstrip()
                if new_text2 != new_text:
                    new_text = new_text2
                    trimmed_tail += 1

        # (5) 길이 ≤ 10자면 drop
        if len(new_text.strip()) <= 10:
            removed_short += 1
            continue

        # 반영
        p = dict(p)
        p['text'] = new_text
        new_paras.append(p)

    doc['paragraphs'] = new_paras

stats = {
    "modified_del_mark_segments": modified_del_marks,
    "removed_empty_after_cut": removed_empty_after_cut,
    "removed_no_sentence": removed_no_sentence,
    "trimmed_tail": trimmed_tail,
    "removed_short_texts": removed_short
}
print("Stats:", stats)


In [ ]:

def save_json(obj, path: Path):
    with path.open('w', encoding='utf-8') as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

save_json(data, OUTPUT_PATH)
print('Saved:', OUTPUT_PATH)


In [ ]:

# 빠른 검증: 정의/목록 스타일들이 살아 있는지 훑어보기
examples = []
targets = [
    "이 기준서에서 사용하는 용어의 정의는 다음과 같다",
    "다음과 같다.",
    "⑴", "①", "㉠", ":"
]
for doc in data.get("documents", []):
    for p in doc.get("paragraphs", []):
        t = p.get("text","")
        if any(x in t for x in targets):
            examples.append({"std": doc.get("standard_no"),
                             "para_id": p.get("para_id"),
                             "page": p.get("page"),
                             "text": (t[:160] + "…") if len(t)>160 else t})
        if len(examples) >= 10:
            break
    if len(examples) >= 10:
        break

print("Sample examples (max 10):")
for e in examples:
    print(f"- [{e['std']}:{e['para_id']}] p.{e['page']} :: {e['text']}")
